In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 전처리

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Reshape, SeparableConv2D, Conv2D, BatchNormalization, Multiply, Layer,Attention, LayerNormalization, Add
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [3]:
# 데이터 경로
train_data_dir = '/content/drive/MyDrive/Data/img/train'
validation_data_dir = '/content/drive/MyDrive/Data/img/val'
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5853 images belonging to 4 classes.
Found 1168 images belonging to 4 classes.
Found 1137 images belonging to 4 classes.


#MobileNetV1 기반 모델링(실행환경 T4)

In [4]:
# Base model 정의
def create_base_model(input_shape):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    return base_model

# 패치 추출 레이어 정의
def patch_extraction_layer():
    return tf.keras.Sequential([
        SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'),
        SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'),
        Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
    ], name='patch_extraction')

# Pre-classification 레이어 정의
def create_pre_classification_layer():
    return tf.keras.Sequential([
        Dense(32, activation='relu'),
        BatchNormalization()
    ], name='pre_classification')

In [5]:
def create_mobilenet_with_cbam_block(input_shape, num_classes):
    base_model = create_base_model(input_shape)
    inputs = Input(shape=input_shape)
    # 기본 모델
    x = base_model(inputs, training=False)
    # GlobalAveragePooling2D 및 Dropout
    x = GlobalAveragePooling2D(name='gap')(x)
    # 출력층
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# 모델 생성 및 컴파일
model = create_mobilenet_with_cbam_block(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

17225924/17225924 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 gap (GlobalAveragePooling2  (None, 1024)              0         
 D)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 3232964 (12.33 MB)
Trainable params: 4100 (16.02 KB)
Non-trainable params: 3228864 (12.32 MB)
____________________

In [6]:
# 학습률 조정 콜백
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, min_delta=0.005, min_lr=1e-7)

# EarlyStopping 콜백
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.005, restore_best_weights=True)

# 클래스 가중치 추가
train_samples_per_class = np.array([1440, 1489, 1477, 1447])
total_train_samples = np.sum(train_samples_per_class)
class_weights = {i: total_train_samples / (len(train_samples_per_class) * count) for i, count in enumerate(train_samples_per_class)}


In [ ]:
# 모델 훈련
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights
)

Epoch 1/100
183/183 [==============================] - 4396s 24s/step - loss: 1.4283 - accuracy: 0.3214 - val_loss: 1.3999 - val_accuracy: 0.3493 - lr: 0.0010
Epoch 2/100
183/183 [==============================] - 554s 3s/step - loss: 1.2752 - accuracy: 0.4304 - val_loss: 1.2757 - val_accuracy: 0.4127 - lr: 0.0010
Epoch 3/100
183/183 [==============================] - 553s 3s/step - loss: 1.2088 - accuracy: 0.4562 - val_loss: 1.2809 - val_accuracy: 0.4366 - lr: 0.0010
Epoch 4/100
183/183 [==============================] - 552s 3s/step - loss: 1.1712 - accuracy: 0.4856 - val_loss: 1.2382 - val_accuracy: 0.4358 - lr: 0.0010
Epoch 5/100
183/183 [==============================] - 550s 3s/step - loss: 1.1459 - accuracy: 0.5026 - val_loss: 1.2359 - val_accuracy: 0.4503 - lr: 0.0010
Epoch 6/100
183/183 [==============================] - 547s 3s/step - loss: 1.1339 - accuracy: 0.5102 - val_loss: 1.2820 - val_accuracy: 0.4307 - lr: 0.0010
Epoch 7/100
183/183 [==============================] - 5

In [ ]:
# 파인튜닝 과정
base_model = model.layers[1]
base_model.trainable = True

# 사전 학습된 모델의 전체 레이어 갯수
num_layers = len(base_model.layers)

# 훈련되지 않고 고정될 레이어 비율
freeze_ratio = 0.5
num_freeze_layers = int(num_layers * freeze_ratio)

# 하위 레이어 고정, 상위 레이어 해제
for layer in base_model.layers[:num_freeze_layers]:
    layer.trainable = True
for layer in base_model.layers[num_freeze_layers:]:
    layer.trainable = False

# 모델 재컴파일
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 gap (GlobalAveragePooling2  (None, 1024)              0         
 D)                                                              
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 3232964 (12.33 MB)
Trainable params: 273220 (1.04 MB)
Non-trainable params: 2959744 (11.29 MB)
_________________________________________________________________


In [ ]:
# 모델 학습
history_fine = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/100
183/183 [==============================] - 556s 3s/step - loss: 1.0294 - accuracy: 0.5624 - val_loss: 1.0921 - val_accuracy: 0.5368 - lr: 1.0000e-04
Epoch 2/100
183/183 [==============================] - 552s 3s/step - loss: 0.9616 - accuracy: 0.6122 - val_loss: 0.9980 - val_accuracy: 0.5676 - lr: 1.0000e-04
Epoch 3/100
183/183 [==============================] - 542s 3s/step - loss: 0.8844 - accuracy: 0.6422 - val_loss: 0.9297 - val_accuracy: 0.6147 - lr: 1.0000e-04
Epoch 4/100
183/183 [==============================] - 541s 3s/step - loss: 0.8410 - accuracy: 0.6615 - val_loss: 0.8869 - val_accuracy: 0.6361 - lr: 1.0000e-04
Epoch 5/100
183/183 [==============================] - 562s 3s/step - loss: 0.7851 - accuracy: 0.6911 - val_loss: 0.9024 - val_accuracy: 0.6276 - lr: 1.0000e-04
Epoch 6/100
183/183 [==============================] - 540s 3s/step - loss: 0.7650 - accuracy: 0.6972 - val_loss: 0.8731 - val_accuracy: 0.6318 - lr: 1.0000e-04
Epoch 7/100
183/183 [=============

In [ ]:
# 검증 데이터 확인
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

37/37 [==============================] - 75s 2s/step - loss: 0.8397 - accuracy: 0.6661
Validation Loss: 0.8396927118301392
Validation Accuracy: 0.6660959124565125


In [ ]:
# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


36/36 [==============================] - 796s 23s/step - loss: 0.8142 - accuracy: 0.6737
Test Loss: 0.8141984939575195
Test Accuracy: 0.6737027168273926


In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/MobileNetV1_01_03.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Sub

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# 이미지 전처리 함수
def load_and_prepare_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0  # 정규화
    return img_tensor

# 감정을 예측하는 함수
def predict_emotion(model, img_path):
    print("이미지 로딩 중...")
    test_image = load_and_prepare_image(img_path)
    print("감정 예측 중...")
    prediction = model.predict(test_image)
    return prediction

# 모델 경로와 이미지 경로
model_path = '/content/drive/MyDrive/MobileNetV1-6.h5'  # 학습된 모델 경로
image_path = '/content/drive/MyDrive/testimg/5.jpg'  # 테스트할 이미지 경로

# 모델 로드
model = load_model(model_path)

# 예측 결과 얻기
predictions = predict_emotion(model, image_path)
emotion_index = np.argmax(predictions)
emotion_labels = ['화남', '행복', '슬픔', '충격']  # 실제 감정 레이블로 변경

print("예측된 감정:", emotion_labels[emotion_index])
print("각 감정의 비율:")
for i, label in enumerate(emotion_labels):
    print(f"{label}: {predictions[0][i] * 100:.2f}%")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# 모델 파일 경로
model_path = '/content/drive/MyDrive/MobileNetV1_01.h5'

# 모델 로드
model = load_model(model_path)

# 테스트 데이터 디렉토리 경로
test_data_dir = '/content/drive/MyDrive/Data/img/test'

# 테스트 데이터를 위한 ImageDataGenerator 생성
val_test_datagen = ImageDataGenerator(rescale=1./255)

# 테스트 데이터 생성기 생성
test_generator = val_test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 전체 검증
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 예측 값 얻기
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 각 클래스별 정확도 계산 및 잘못 예측한 경우 정리
misclassified_summary = []

for i, label in enumerate(class_labels):
    indices = np.where(true_classes == i)[0]
    class_accuracy = accuracy_score(true_classes[indices], predicted_classes[indices])
    print(f"Accuracy for class {label}: {class_accuracy * 100:.2f}%")

    # 잘못 예측한 경우 저장
    misclassified_indices = indices[true_classes[indices] != predicted_classes[indices]]
    for index in misclassified_indices:
        true_label = class_labels[true_classes[index]]
        predicted_label = class_labels[predicted_classes[index]]
        misclassified_summary.append((true_label, predicted_label))

# DataFrame으로 정리
misclassified_df = pd.DataFrame(misclassified_summary, columns=["True Label", "Predicted Label"])

# 잘못 예측한 경우를 집계
misclassified_counts = misclassified_df.groupby(['True Label', 'Predicted Label']).size().reset_index(name='Count')

# 정리된 결과 출력
print("\nMisclassified Cases Summary:")
print(misclassified_counts)


36/36 [==============================] - 81s 2s/step - loss: 0.6140 - accuracy: 0.7581
Test Loss: 0.6139590740203857
Test Accuracy: 0.7581354379653931
